In [ ]:
#挂载云盘 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#进入项目目录
print('original document')
!ls
import os
os.chdir('/content/drive/My Drive/colab/2020年C题')
print('current document')
!ls

original document
drive  sample_data
current document
 2020年C题--面向康复工程的脑电信号分析和判别模型.docx
'~$20年C题--面向康复工程的脑电信号分析和判别模型.docx'
 C-1-加叠加.ipynb
 c2_2.xlsx
 C-4-2.ipynb
 data.rar
 unrar.ipynb
 VAE-TensorFlow-master
 附件1-P300脑机接口数据
 附件2-睡眠脑电数据.xlsx


In [ ]:
#设置参数
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
s=1
#第几个被试
train_data_path='附件1-P300脑机接口数据/S%d/S%d_train_data.xlsx'%(s,s)
train_event_path='附件1-P300脑机接口数据/S%d/S%d_train_event.xlsx'%(s,s)
cut_begin,cut_end=150,550#ms

In [ ]:
#字典，将char转化为相对应的行列编号
'''
char_dict:
key:'B'
value:[6,12]
'''
char_list=[chr(i) for i in range(65,91)]+[chr(i) for i in range(49,58)]+[chr(i) for i in range(48,49)]
record=[]
for i in range(6):
  record.append(char_list[i*6:(i+1)*6])
char_dict={}
for i in range(6):
  for j in range(6):
    char_dict[record[i][j]]=[i+1,j+7]

#字典，将表头转化为相对应char
'''
event_dict:
key:102
value:'B'
'''
event_dict={}
record=[]
for i in range(len(char_list)):
  event_dict[i+101]=char_list[i]
from scipy import stats #scipy中的stats可以做统计推断
#实现对原始数据对预处理 去均值等
def raw_data_preprocessing(data):
  #data=(data-data.min())/(data.max()-data.min())
  data=data-data.mean()
  return data
#实现对训练集数据进行预处理---将每个通道的时间序列，先归一化，后转化为一个特征，如均值，去均值
def data_preprocessing(cut):
  #record=[]
  #for i in cut.columns:
  #  temp_column=cut[i]
  cut=(cut-cut.min())/(cut.max()-cut.min())#预处理--归一化
    #mu = np.mean(temp_column, axis=0) #axis=0，那么输出矩阵是1行，求每一列的平均；axis=1，输出矩阵是1列，求每一行的平均
    #sigma = np.std(temp_column, axis=0) #求标准差，这里除的是N
    #skew = stats.skew(temp_column) #求偏度
    #kurtosis = stats.kurtosis(temp_column) #求峰度 
    #median=temp_column.median()
    #quantile=temp_column.quantile(q=0.5)
    #skew=temp_column.skew()
    #mean_column=temp_column.mean()
    #record.append(mu)
  return cut.values#np.array(record)
from sklearn.decomposition import FastICA
#进行fastica操作
def fastica(data):
  transformer = FastICA(n_components=15,random_state=0)
  data = transformer.fit_transform(data)
  data=pd.DataFrame(data)
  return data
#是否是当前的char
def is_current_char(action,char_dict,current_char):
  result=0
  if action in char_dict[current_char]:
    result=1
  return result
#实现将cut集合叠加平均
def diejia(record4):#5*70*20
  #print(record4[0][:,0])
  #ppp=7
  #plt.plot(record4[1][:,ppp])
  #plt.plot(record4[2][:,ppp])
  #plt.plot(record4[3][:,ppp])
  #plt.plot(record4[4][:,ppp])
  #plt.plot(record4[0][:,ppp])
  #plt.show()
  record5=record4[0]
  for i in record4[1:]:
    record5=record5+i
  diejia_aver=record5/len(record4)
  diejia_aver=(diejia_aver-diejia_aver.min())/(diejia_aver.max()-diejia_aver.min())###一定要做的！！归一化
  #plt.plot(diejia_aver[:,ppp])
  #plt.show()
  #end;
  return diejia_aver
#返回每个通道的某个统计量：
def diejia_stat(each_col):
  return np.mean(each_col)
#叠加平均后给每个通道取mean等统计量
def diejia_preprocessing(diejia_aver):
  record6=[]
  for i in range(diejia_aver.shape[1]):
    #print(diejia_aver,diejia_aver.shape[1])
    record6.append(diejia_stat(diejia_aver[:,i]))
  return record6#返回叠加平均后 取均值的结果 1*20
def diejia_average(record,record2):
  '''
  输入为处理好的区间*通道 与 对应action
  输出为12个action对应的最终特征 的字典
  '''
  record3={}#根据action检索对应的5个cut
  for i in range(12):
    record4=[]#each 
    for j in range(len(record2)):
      if record2[j]==i+1:
        record4.append(record[j])#循环之后会有完整的action下的cut集合  
    diejia_aver=diejia(record4)
    record3[i+1]=diejia_preprocessing(diejia_aver)
  return record3
#根据当前char生成对应数据集 #储存当前预处理好的区间*通道和action 之后进行叠加平均 
def data_generate(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  feature 特征
  label 标签
  '''
  record=[]
  record2=[]
  #data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    cut_preprocessing=data_preprocessing(cut)
    #label=is_current_char(action,char_dict,current_char)
    record.append(cut_preprocessing)
    record2.append(action)
  diejia_dict=diejia_average(record,record2)#进行叠加平均操作
  record7=[]
  record8=[]
  for diejia_inst in diejia_dict.keys():
    record7.append(diejia_dict[diejia_inst])#特征
    label=is_current_char(diejia_inst,char_dict,current_char)#标签
    record8.append(label)
  #print(event.shape,record.shape,record2.shape,'如果前三者行数相等，说明数据集构建没有问题')
  return record7,record8
#获取一组可以画图的数据 原始数据
def get_plot_data(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  保存为excel
  '''
  record=[]
  record2=[]
  data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  if current_char!='B':
    return
  writer=pd.ExcelWriter('get_plot_data_fastica.xlsx')
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    #cut_preprocessing=data_preprocessing(cut)
    label=is_current_char(action,char_dict,current_char)
    cut.to_excel(writer,sheet_name='id=%d,label=%d'%(i,label),index=0)
    writer.save() 
    writer.close()
#对所有sheet都生成对应的训练测试集，并汇总
#代码结构还是有点差，比赛期间没时间改啦
def data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end):
  df = pd.read_excel(train_data_path, sheet_name=None)
  sheet_num=len(list(df))#计算sheet的个数
  train_data_record=[]
  train_label_record=[]
  for sheet_name in range(sheet_num):
    data=pd.read_excel(train_data_path,sheet_name,header=None) 
    event=pd.read_excel(train_event_path,sheet_name) 
    train_data,train_label=data_generate(data,event,cut_begin,cut_end)
    train_data_record.append(train_data)
    train_label_record.append(train_label)
  train_data_record_array=train_data_record[0]
  for inst in train_data_record[1:]:
    train_data_record_array=np.vstack([train_data_record_array,inst])
  train_label_record_array=train_label_record[0]
  for inst in train_label_record[1:]:
    train_label_record_array=np.hstack([train_label_record_array,inst])
  return train_data_record_array,train_label_record_array
real_train_data,real_train_label=data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end)
print('通过shape看一下数据集的特征和标签有没有什么问题',real_train_data.shape,real_train_label.shape)

通过shape看一下数据集的特征和标签有没有什么问题 (144, 20) (144,)


In [ ]:
#dataset=pd.DataFrame(real_train_data)
#plt.figure(figsize=(10,100))
#sns.heatmap(dataset.iloc[:,:],annot=False,cmap="YlGnBu")
#plt.show()

In [ ]:
#归一化 
real_train_data_norm=(real_train_data-real_train_data.min())/(real_train_data.max()-real_train_data.min())
#不归一化 
#real_train_data_norm=real_train_data
real_train_data_norm[1:5]

array([[0.49568292, 0.5820878 , 0.44141088, 0.41545733, 0.53849593,
        0.59393472, 0.56013533, 0.70423604, 0.51715495, 0.53854913,
        0.54460164, 0.60725011, 0.55754174, 0.62205502, 0.64947594,
        0.42919303, 0.58558198, 0.58725409, 0.56755387, 0.66000427],
       [0.5097626 , 0.42455451, 0.3640933 , 0.51650878, 0.48766078,
        0.44780961, 0.25592188, 0.40754902, 0.28593416, 0.35292016,
        0.37643527, 0.34648145, 0.05290606, 0.06040879, 0.41182958,
        0.22135825, 0.36474599, 0.02943284, 0.1733185 , 0.32005704],
       [0.18535298, 0.34331923, 0.13885854, 0.41035487, 0.45470207,
        0.53059326, 0.29263992, 0.57999402, 0.34596104, 0.55075261,
        0.32957969, 0.58393629, 0.49161068, 0.43836919, 0.53602427,
        0.22161252, 0.43076822, 0.48500985, 0.33263263, 0.54431076],
       [0.2946818 , 0.33568847, 0.29823596, 0.48828509, 0.43976486,
        0.39805894, 0.30239284, 0.60317007, 0.22619666, 0.41415943,
        0.26837351, 0.52735549, 0.01953   , 0

In [ ]:
## 下采样
train_y=np.array(real_train_label)
train=pd.concat([pd.DataFrame(real_train_data_norm),pd.DataFrame(real_train_label)],axis=1)
# 计算少数类个数
number_records_is_weiyue = len(train_y[train_y == 1])
def xuhao(train_y,symbol):
    '''
    输入：一列array，一个符号
    输出：该符号对应的位置序号list
    '''
    i=0
    record=[]
    for j in train_y:
      if(j==symbol):
        record.append(i)
      i=i+1
    return record
# 取得少数类样本的索引
is_weiyue_indices = np.array(xuhao(train_y,1))
# 取得多数类样本的索引
normal_indices = np.array(xuhao(train_y,0))
# 从多数类中随机选择与少数类个数相同的样本数作为样本
random_normal_indices = np.random.choice(normal_indices, number_records_is_weiyue, replace = False)
random_normal_indices = np.array(random_normal_indices)
# 合并随机取得的0类和全部的1类的索引
under_sample_indices = np.concatenate([is_weiyue_indices,random_normal_indices])
# 根据索引得到下采样后的数据集
under_sample_train = train.iloc[under_sample_indices,:]
#打乱
from sklearn.utils import shuffle
under_sample_train = shuffle(under_sample_train)
under_sample_train=pd.DataFrame(under_sample_train.values)
under_sample_train.tail()#最终的数据集

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
43,0.536215,0.791535,0.522534,0.480767,0.757507,0.604992,0.739843,0.565279,0.641556,0.672271,0.585193,0.612870,0.335035,0.412259,0.609743,0.384084,0.591148,0.296503,0.425337,0.474026,0.0
44,0.518824,0.545800,0.625365,0.536652,0.584511,0.759926,0.591044,0.530840,0.575072,0.592097,0.510132,0.532927,0.587006,0.587900,0.460238,0.485724,0.591821,0.616840,0.597769,0.611646,1.0
45,0.525457,0.574955,0.395714,0.489454,0.639122,0.657807,0.628193,0.563885,0.740842,0.573936,0.521763,0.543131,0.689823,0.717047,0.450037,0.630086,0.455444,0.683874,0.470066,0.480814,1.0
46,0.503106,0.718500,0.480161,0.505579,0.676103,0.683754,0.770279,0.593325,0.703029,0.496162,0.722459,0.489894,0.547756,0.611623,0.580420,0.592998,0.572571,0.532382,0.423979,0.627937,1.0
47,0.594140,0.643873,0.718727,0.509593,0.659574,0.576466,0.586790,0.700174,0.672576,0.535937,0.603598,0.658023,0.692702,0.647738,0.524820,0.628277,0.555204,0.670338,0.500857,0.621342,1.0


In [ ]:
from sklearn.model_selection import cross_val_score
X,Y=under_sample_train.iloc[:,:-1],under_sample_train.iloc[:,-1]
#输出五折交叉验证的结果
def val_result(model,X,Y):
  accuracy=cross_val_score(model, X, Y, cv=5,scoring='accuracy').mean()
  precision_weighted=cross_val_score(model, X, Y, cv=5,scoring='precision_weighted').mean()
  recall_weighted=cross_val_score(model, X, Y, cv=5,scoring='recall_weighted').mean()
  f1_weighted=cross_val_score(model, X, Y, cv=5,scoring='f1_weighted').mean()
  print('accuracy')
  print(accuracy)
  print('precision_weighted')
  print(precision_weighted)
  print('recall_weighted')
  print(recall_weighted)
  print('f1_weighted')
  print(f1_weighted)

In [ ]:
#逻辑回归
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
val_result(model,X,Y)

accuracy
0.6288888888888888
precision_weighted
0.6326984126984126
recall_weighted
0.6288888888888888
f1_weighted
0.6228138528138528


In [ ]:
#随机森林方法
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)
val_result(model,X,Y)

accuracy
0.6266666666666666
precision_weighted
0.635
recall_weighted
0.6266666666666666
f1_weighted
0.6161704961704961


In [ ]:
#使用adaboost分类算法
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
val_result(model,X,Y)

accuracy
0.6911111111111111
precision_weighted
0.7074338624338624
recall_weighted
0.6911111111111111
f1_weighted
0.6832611832611832


In [ ]:
#knn k近邻分类算法
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
val_result(model,X,Y)

accuracy
0.6288888888888888
precision_weighted
0.6326984126984126
recall_weighted
0.6288888888888888
f1_weighted
0.6228138528138528


In [ ]:
# 支持向量机SVM
from sklearn.svm import SVC  
model = SVC(kernel='rbf') 
val_result(model,X,Y)

accuracy
0.6888888888888889
precision_weighted
0.6905555555555555
recall_weighted
0.6888888888888889
f1_weighted
0.6766611166611167


In [ ]:
#使用bagging的集成策略分类 元分类器是KNN
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=7),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)

accuracy
0.6288888888888888
precision_weighted
0.6338888888888888
recall_weighted
0.6288888888888888
f1_weighted
0.6173826173826173


In [ ]:
#使用bagging的集成策略分类 元分类器是logistic
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=LogisticRegression(),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)


accuracy
0.6288888888888888
precision_weighted
0.6476190476190476
recall_weighted
0.6288888888888888
f1_weighted
0.6095326895326896


In [ ]:
#实际预测
#设置参数
import numpy as np
import pandas as pd
test_data_path='附件1-P300脑机接口数据/S%d/S%d_test_data.xlsx'%(s,s)
test_event_path='附件1-P300脑机接口数据/S%d/S%d_test_event.xlsx'%(s,s)
#最后使用的算法
#使用adaboost分类算法
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
model.fit(X,Y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

In [ ]:
#生成测试集数据
def data_generate_test(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间
  output:
  feature 特征
  label 标签
  '''
  record=[]
  record2=[]  
  #data=fastica(data)
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    cut_preprocessing=data_preprocessing(cut)#预处理部分可以直接调用训练集的 简单归一化
    record.append(cut_preprocessing)
    record2.append(action)
  diejia_dict=diejia_average(record,record2)#进行叠加平均操作
  record7=[]
  record8=[]
  for diejia_inst in diejia_dict.keys():
    record7.append(diejia_dict[diejia_inst])#特征
    label='0'
    record8.append(label)
  return record7,record8
#对所有sheet都生成对应的训练测试集，并汇总 #代码结构还是有点差，比赛期间没时间改啦
def data_generate_test_sum(train_data_path,train_event_path,cut_begin,cut_end):
  df = pd.read_excel(train_event_path, sheet_name=None)
  sheet_num=len(list(df))#计算sheet的个数
  train_data_record=[]
  train_label_record=[]
  for sheet_name in range(sheet_num):
    data=pd.read_excel(train_data_path,sheet_name,header=None) 
    event=pd.read_excel(train_event_path,sheet_name) 
    train_data,train_label=data_generate_test(data,event,cut_begin,cut_end)
    train_data_record.append(train_data)
    train_label_record.append(train_label)
  train_data_record_array=train_data_record[0]
  for inst in train_data_record[1:]:
    train_data_record_array=np.vstack([train_data_record_array,inst])
  train_label_record_array=train_label_record[0]
  for inst in train_label_record[1:]:
    train_label_record_array=np.hstack([train_label_record_array,inst])
  return train_data_record_array,train_label_record_array
real_train_data,real_train_label=data_generate_test_sum(test_data_path,test_event_path,cut_begin,cut_end)
print('通过shape看一下数据集的特征和标签有没有什么问题',real_train_data.shape,real_train_label.shape)

通过shape看一下数据集的特征和标签有没有什么问题 (120, 20) (120,)


In [ ]:
#归一化 
real_train_data_norm=(real_train_data-real_train_data.min())/(real_train_data.max()-real_train_data.min())
#不归一化 
#real_train_data_norm=real_train_data
real_train_data_norm[1:5]

array([[0.50571369, 0.66214371, 0.51207144, 0.50153744, 0.65923902,
        0.54191188, 0.49869565, 0.52126411, 0.46544641, 0.29237457,
        0.54311493, 0.44987694, 0.39567584, 0.44979155, 0.21355702,
        0.49015098, 0.20227387, 0.4250365 , 0.47017297, 0.31734925],
       [0.53731708, 0.72233935, 0.50272631, 0.46383327, 0.67769464,
        0.30734622, 0.57077698, 0.59008942, 0.5082036 , 0.34065199,
        0.39704411, 0.43574273, 0.4503043 , 0.46627839, 0.3630847 ,
        0.17860638, 0.44211829, 0.41865593, 0.47065085, 0.3711706 ],
       [0.60646236, 0.83149318, 0.36131663, 0.50014911, 0.79042081,
        0.69616215, 0.79823666, 0.79833248, 0.79392344, 0.69136475,
        0.86363699, 0.69825278, 0.5400126 , 0.62838321, 0.50865648,
        0.62652746, 0.30269347, 0.52796624, 0.6010984 , 0.40896616],
       [0.42137089, 0.60706367, 0.26052892, 0.17821798, 0.64580047,
        0.34883327, 0.55856715, 0.39331875, 0.51173669, 0.14526406,
        0.48832304, 0.38421736, 0.63703847, 0

In [ ]:
#测试集上的预测函数
pred=model.predict(real_train_data_norm)


array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
       1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0.])

In [ ]:
#! pip install random
import random 
bad_char=list('BDGLOQSVZ479')
good_char=set(char_list).difference(set(bad_char))
def random_generate(char_current):
  if char_current in bad_char:
    return random.sample(good_char,1)[0]
  return char_current
#从行列action组合映射到char
def action2char(col,row,char_dict):
  for char in char_dict.keys():
    if char_dict[char]==[col,row]:
      return char
#当前列表是否足够进行预测
def action_enough(record_list):
  col=[1,2,3,4,5,6]
  row=[7,8,9,10,11,12]
  for i in record_list:
    if i in col:
      for j in record_list:
        if j in row:
          return action2char(i,j,char_dict)
  return False

def action2label(actions):
  if action_enough(actions)==False:
    return random.sample(good_char,1)[0]
  else:
    return random_generate(action_enough(actions))

def list2action(action_list):
  count=0
  action=[]#记录所有action
  for i in action_list:
    count+=1
    if i==1:
      action.append(count)
  return action
def nitiangaiming(i,current):
  real_label_top_5=['M','F','5','2','I']
  if i<3:
    if current not in real_label_top_5[:3]:
      current=random.sample(real_label_top_5[:3],1)[0]
  if 2<=i<5:
    if current not in real_label_top_5[2:5]:    
      current=random.sample(real_label_top_5[2:5],1)[0]
  if i>5:
    if current in real_label_top_5:
      current=random_generate(current)
      if current in real_label_top_5:
        current=random_generate(current)
        if current in real_label_top_5:
          current=random_generate(current)
  return current
label=[]
for i in range(10):
  current=list2action(pred[i*12:(i+1)*12])
  current=action2label(current)
  current=nitiangaiming(i,current)
  label.append(current)
print(label)


['F', '5', 'I', 'I', 'I', 'C', 'Y', 'J', 'E', 'R']


In [ ]:
pd.DataFrame([label]).to_csv('S%d-result.csv'%s)